In [1]:
import pandas as pd
import numpy as np
import glob

## Unión de ficheros y formateo

In [2]:
files = glob.glob('./datos/original/oferta/*.csv')

cols = list(pd.read_csv(files[0], sep=';',nrows=1))
# cols = list(filter(lambda col: col.strip(), cols))
dfs = []
dfs.append(pd.read_csv(files[0],header=0,sep=';',encoding='utf-8',dtype={'Elinea':'str'}))
dfs.append([pd.read_csv(f, header=0, sep=";",encoding="iso8859-1",dtype={'Elinea':'str'}) for f in files[1:]][0])
oferta = pd.concat(dfs,ignore_index=True)

In [3]:
del dfs

In [4]:
oferta['FServicio'] = pd.to_datetime(oferta['FServicio'],format='%d/%m/%Y')

oferta['year'] = oferta['FServicio'].dt.year 
oferta['month'] = oferta['FServicio'].dt.month
oferta['month_name'] = oferta['FServicio'].dt.month_name()
oferta['week'] = oferta['FServicio'].dt.isocalendar().week

In [5]:
tipo_dia = pd.read_csv('./datos/day_calendar.csv')
tipo_dia['date'] = pd.to_datetime(tipo_dia['date'],dayfirst=True)
tipo_dia['day_name'] = tipo_dia['date'].dt.day_name()
tipo_dia = tipo_dia.set_index(['date'])

In [6]:
tipo_dia = pd.read_csv('./datos/original/Calendario_EMT_2019-2022-EXPTE.213-2022-01149.csv',sep=';')
tipo_dia['Fecha'] = pd.to_datetime(tipo_dia['Fecha'],dayfirst=True)
tipo_dia['day_name'] = tipo_dia['Fecha'].dt.day_name()
tipo_dia = tipo_dia.set_index(['Fecha'])

In [7]:
oferta = oferta.join(tipo_dia,on='FServicio')

In [8]:
oferta

,CLinea,Elinea,Denominacion,FServicio,IdFranja,Intervalo,Coches,year,month,month_name,week,TipoDiaMo,TemporadaTG,DiaSemana,day_name
0,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H07,070000 - 075959,2.0,2019,1,January,1,FE,IV,M,Tuesday
1,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H08,080000 - 085959,3.0,2019,1,January,1,FE,IV,M,Tuesday
2,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H09,090000 - 095959,3.0,2019,1,January,1,FE,IV,M,Tuesday
3,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H10,100000 - 105959,4.0,2019,1,January,1,FE,IV,M,Tuesday
4,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019-01-01,H11,110000 - 115959,4.0,2019,1,January,1,FE,IV,M,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978841,766,SE766,ATOCHA - POLIGONO INDUSTRIAL DE VALLECAS,2022-01-30,H22,22:00 - 22:59,1.0,2022,1,January,4,FE,IV,D,Sunday
3978842,766,SE766,ATOCHA - POLIGONO INDUSTRIAL DE VALLECAS,2022-01-31,H08,08:00 - 08:59,1.0,2022,1,January,5,LA,IV,L,Monday
3978843,766,SE766,ATOCHA - POLIGONO INDUSTRIAL DE VALLECAS,2022-01-31,H09,09:00 - 09:59,1.0,2022,1,January,5,LA,IV,L,Monday
3978844,766,SE766,ATOCHA - POLIGONO INDUSTRIAL DE VALLECAS,2022-01-31,H21,21:00 - 21:59,1.0,2022,1,January,5,LA,IV,L,Monday


## Cálculo de medias

In [9]:
tmp = oferta.copy()

In [10]:
# https://stackoverflow.com/questions/38869778/pandas-set-column-equal-to-grouped-sum-of-another-column?noredirect=1&lq=1
tmp['MediaCochesMes'] = tmp.groupby(by=['CLinea', 'year','month', 'IdFranja']).Coches.transform('mean')

tmp['MediaCochesMes_Laborables'] = tmp[tmp['TipoDiaMo']=='LA'].groupby(by=['CLinea','IdFranja','year','month']).Coches.transform('mean')

In [11]:
meses_oferta = oferta[['CLinea','Elinea','Denominacion','year','month','IdFranja']].drop_duplicates().reset_index(drop=True).copy()

#Unión con el dataframe temporal para obtener las columnas de medias al mes
meses_oferta = meses_oferta.join(tmp[['CLinea','year','month','IdFranja','MediaCochesMes']].drop_duplicates(subset=['CLinea','year','month','IdFranja','MediaCochesMes']).set_index(keys=['CLinea','year','month','IdFranja']),on=['CLinea','year','month','IdFranja'])
meses_oferta = meses_oferta.join(tmp.loc[(tmp['MediaCochesMes_Laborables'].notna()),['CLinea','year','month','IdFranja','MediaCochesMes_Laborables']].drop_duplicates(subset=['CLinea','year','month','IdFranja','MediaCochesMes_Laborables']).set_index(keys=['CLinea','year','month','IdFranja']),on=['CLinea','year','month','IdFranja'])

In [12]:
meses_oferta.sort_values(by=['CLinea','year','month','IdFranja'])

,CLinea,Elinea,Denominacion,year,month,IdFranja,MediaCochesMes,MediaCochesMes_Laborables
18,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019,1,H06,3.840000,4.000000
0,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019,1,H07,5.580645,6.857143
1,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019,1,H08,6.580645,7.857143
2,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019,1,H09,7.387097,8.857143
3,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019,1,H10,8.903226,10.857143
...,...,...,...,...,...,...,...,...
22600,786,SE,PRINCIPE DE VERGARA - CALLAO,2019,5,H22,5.900000,7.000000
22601,786,SE,PRINCIPE DE VERGARA - CALLAO,2019,5,H23,4.266667,5.000000
22602,786,SE,PRINCIPE DE VERGARA - CALLAO,2019,5,M00,3.633333,4.000000
22603,786,SE,PRINCIPE DE VERGARA - CALLAO,2019,5,M01,3.000000,3.000000


## Hora punta Madrid
Según el informe anual de [2018 de CRTM](https://www.crtm.es/media/712934/edm18_sintesis.pdf#page=54) y la [Encuesta Sintética de Movilidad - ESM14](https://www.crtm.es/media/519661/esm_2014.pdf) las franjas horarias quedan de la siguiente manera:
| Afluencia  | Intervalo horas | Tipo       | Información                                                    |
| ---------  | --------------- | ---------- | -------------------------------------------------------------- |
| Máxima     | 07:00 - 09:00   | Hora punta | Movilidad ocupacional, trabajo y estudios.                     |
| Baja       | 10:00 - 13:00   | Hora valle |                                                                |
| Media      | 13:00 - 15:00   | Hora punta | Movilidad ocupacional, trabajo y estudios.                     |
| Baja       | 15:30 - 16:00   | Hora valle |                                                                |
| Media      | 16:00 - 18:00   | Hora punta | Salida de los centros escolares o el fin de la jornada laboral.|
| Media-Baja | 18:00 - 23:00   | Decreciente| Salida de los centros escolares o el fin de la jornada laboral.|


_Los viajes por movilidad ocupacional, trabajo y estudios, se producen en dos periodos concretos, entre las 6h y las 9h de la mañana y entre la 13h y las 15 h de la tarde. Existe otro pico entre las 16h y las 18h relacionado con la salida de los centros escolares o el fin de la jornada laboral._


![Distribución horaria de los viajeros según su actividad](./images/markdown/distribución_horaria_viajeros_actividad.png)

### Cálculo de coches al día en "hora punta"

In [13]:
HORAS_PUNTA = ['H07','H08','H09']

In [14]:
tmp['MediaCochesDía_HoraPunta'] = tmp[(tmp['IdFranja'].isin(HORAS_PUNTA)) & (tmp['TipoDiaMo']=="LA")].groupby(by=['CLinea', 'FServicio'],as_index=False)['Coches'].transform('mean')

In [15]:
horapunta = tmp[tmp['MediaCochesDía_HoraPunta'].notna()].drop_duplicates(subset=['CLinea','FServicio']).copy()

In [16]:
horapunta = horapunta.assign(Coches=horapunta['MediaCochesDía_HoraPunta'],Intervalo='070000 - 095959',IdFranja='HoraPunta')
horapunta = horapunta.drop(columns=['MediaCochesDía_HoraPunta'])

In [17]:
horapunta['MediaCochesMes'] = horapunta.groupby(by=['CLinea', 'year','month']).Coches.transform('mean')
horapunta['MediaCochesMes_Laborables'] = horapunta[horapunta['TipoDiaMo']=='LA'].groupby(by=['CLinea','year','month']).Coches.transform('mean')

In [18]:
meses_oferta = pd.concat([meses_oferta,horapunta[['CLinea','Elinea','Denominacion','year','month','IdFranja','MediaCochesMes','MediaCochesMes_Laborables']]])

In [19]:
meses_oferta = meses_oferta.drop_duplicates(subset=['CLinea','year','month','IdFranja']).sort_values(by=['CLinea','year','month','IdFranja'])

In [20]:
meses_oferta

,CLinea,Elinea,Denominacion,year,month,IdFranja,MediaCochesMes,MediaCochesMes_Laborables
18,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019,1,H06,3.840000,4.000000
0,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019,1,H07,5.580645,6.857143
1,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019,1,H08,6.580645,7.857143
2,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019,1,H09,7.387097,8.857143
3,1,1,PLAZA DE CRISTO REY - PROSPERIDAD,2019,1,H10,8.903226,10.857143
...,...,...,...,...,...,...,...,...
22601,786,SE,PRINCIPE DE VERGARA - CALLAO,2019,5,H23,4.266667,5.000000
638862,786,SE,PRINCIPE DE VERGARA - CALLAO,2019,5,HoraPunta,7.000000,7.000000
22602,786,SE,PRINCIPE DE VERGARA - CALLAO,2019,5,M00,3.633333,4.000000
22603,786,SE,PRINCIPE DE VERGARA - CALLAO,2019,5,M01,3.000000,3.000000


In [21]:
oferta.to_pickle(path='./datos/output/oferta.pkl')
meses_oferta.to_pickle(path='./datos/output/meses_oferta.pkl')

oferta.to_csv('./datos/output/oferta.csv',encoding='utf-8',index=False)
meses_oferta.to_csv('./datos/output/meses_oferta.csv',encoding='utf-8',index=False)

In [22]:
meses_oferta[meses_oferta['CLinea']==25]

,CLinea,Elinea,Denominacion,year,month,IdFranja,MediaCochesMes,MediaCochesMes_Laborables
2692,25,25,PLAZA DE ESPAÑA - CASA DE CAMPO,2019,1,H05,1.000000,1.000000
2693,25,25,PLAZA DE ESPAÑA - CASA DE CAMPO,2019,1,H06,8.240000,9.047619
2674,25,25,PLAZA DE ESPAÑA - CASA DE CAMPO,2019,1,H07,10.709677,13.952381
2675,25,25,PLAZA DE ESPAÑA - CASA DE CAMPO,2019,1,H08,12.838710,16.904762
2676,25,25,PLAZA DE ESPAÑA - CASA DE CAMPO,2019,1,H09,13.451613,16.904762
...,...,...,...,...,...,...,...,...
136733,25,25,PLAZA DE ESPAÑA - CASA DE CAMPO,2022,1,H21,8.225806,8.850000
136734,25,25,PLAZA DE ESPAÑA - CASA DE CAMPO,2022,1,H22,6.806452,7.000000
136735,25,25,PLAZA DE ESPAÑA - CASA DE CAMPO,2022,1,H23,5.774194,6.000000
3880050,25,25,PLAZA DE ESPAÑA - CASA DE CAMPO,2022,1,HoraPunta,15.583333,15.583333
